# Error Handling

## The `panic` macro
If the program fails in such a way that it's state is unrecoverable or you cannot handle the error gracefully, you can call the panic macro and the program will quit immediately with the provided error statement.

In [7]:
panic!();

thread '<unnamed>' panicked at 'explicit panic', src/lib.rs:22:1
stack backtrace:
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [8]:
panic!("crash and burn");

thread '<unnamed>' panicked at 'crash and burn', src/lib.rs:22:1
stack backtrace:
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [12]:
use std::env;
env::set_var("RUST_BACKTRACE", "full");

To see backtrace output, either set the env in the main file, or put BACKTRACE=full beofore cargo

In [13]:
fn d(x: u8){
    if x > 3 {
     panic!("number too large");
    }
}
fn c(x: u8){
    d(x);
}
fn b(x: u8){
    c(x);
}
fn a(x: u8){
    b(x);
}
let x: u8 = 4;
a(x);

// jupyter will not print backtract;
// to see the backtrace run the following command in your terminal;

thread '<unnamed>' panicked at 'number too large', src/lib.rs:15:6
stack backtrace:
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


```sh
RUST_BACKTRACE=1 cargo run
```

## The `Result` enum

In [14]:
enum Result<T, E> {
    Ok(T),
    Err(E),
}

### about
- We use this enum when we want to handle errors gracefully and don't want to crash the program.
- It is simmilar to the option enum, this enum represents 2 variants, success and error.
- This enum is very common in rust so it is also brought into the scope by default.
- unwrap --> this method returns the resultant value in case of success. But if error, it panicks!

In [5]:
use std::{ fs::File, io::{ ErrorKind, prelude::* } };
let path: &str = ".\\src\\main.rs";

// this open method returns a result type,
// because opening a file may fail in cases like file does not exist;
let f = File::open(&path);

let mut f = match f {

    // if succesfully read, return file;
    Ok(file) => file,

    // if failed, match what kind of err occured;
    Err(err) => match err.kind() {

        // if file not found err, then create a file;
        ErrorKind::NotFound => match File::create(&path){

            // if file successfully created, return the file;
            Ok(new_file) => new_file,

            // else panic!
            Err(create_err) => panic!("err finding or creating file: {} | error: {}", &path, create_err)

        },
        // else panic!
        other_error => panic!("problm opening file: {} | error: {}", &path, other_error)
    }
};

let mut buffer = String::new();
f.read_to_string(&mut buffer);

println!("{}", buffer);

extern crate unicode_segmentation;
use unicode_segmentation::UnicodeSegmentation;

fn main() {
    notebook_7_string_to_grapheme();
}

fn notebook_7_string_to_grapheme() {
    let hello: String = String::from("नमस्ते");
    // to print graphemes
    print!("[");
    for g in hello.graphemes(true) {
        print!("{}, ", g);
    }
    println!("\x08\x08]");
}



### Error propagation, `?` operator

```rs
    fn open_file(path: &str) -> Result<File, io::Error> {
        File::open(path)?
    }
```
> **Note:** Err propagation does not work in the main function because you can return the `?` operator only in functions that return Result or Option buy main function returns neither.
If the file exists and is read succesfully, it returns the file. But in case some error occurs, instead of panicking, it returns the err to the caller function.
- Let's write code from above in a simplified manner.

> **Note:** There is a better way to do this without all this nesting, using `closures`. It is in upcoming modules.

In [18]:
use std::{
    fs::{ self, File },
    io::{ self, Read, prelude::* }
};

fn open_file(path: &str) -> Result<String, io::Error> {
    // method 1: read file, store in buffer, return buffer;
    // let mut file = File::open(path)?;
    // let mut buffer: String = String::new();
    // file.read_to_string(&mut buffer)?;
    // Ok(buffer)

    // method 2: method 1 with chaining;
    // chaining the method calls with ? operator;
    // let mut buffer = String::new();
    // File::open(path)?.read_to_string(&mut buffer)?;
    // Ok(buffer)

    // method 3: use fs::read_to_string;
    // this method directly returns the contents of the file as a string;
    fs::read_to_string(path)
}


println!("{:?}", open_file(path).unwrap());


"extern crate unicode_segmentation;\nuse unicode_segmentation::UnicodeSegmentation;\n\nfn main() {\n    notebook_7_string_to_grapheme();\n}\n\nfn notebook_7_string_to_grapheme() {\n    let hello: String = String::from(\"नमस\u{94d}त\u{947}\");\n    // to print graphemes\n    print!(\"[\");\n    for g in hello.graphemes(true) {\n        print!(\"{}, \", g);\n    }\n    println!(\"\\x08\\x08]\");\n}\n"
